In [12]:
import os
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, classification_report
from transformers import DistilBertModel, DistilBertTokenizerFast
import torch
from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)
from datasets import Dataset
from sklearn.metrics import precision_recall_fscore_support
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
import torch
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
df = pd.read_csv('/content/drive/MyDrive/Grammar_autocorrection_df1_new.csv')

# DistilBERT

In [15]:
# # ── 0) Configuration ────────────────────────────────────────────────────────
# MODEL_DIR = 'distilbert_saved_model'

# # ── 1) If already saved, just reload ────────────────────────────────────────
# if os.path.isdir(MODEL_DIR):
#     tokenizer = DistilBertTokenizerFast.from_pretrained(MODEL_DIR)
#     model     = DistilBertForSequenceClassification.from_pretrained(MODEL_DIR)
#     print(f"Loaded existing model from '{MODEL_DIR}', skipping training.")

# # ── 2) Otherwise: 5-fold CV with metrics + retrain full head ────────────────
# else:
#     # Prepare texts & labels
#     texts  = df['Ungrammatical Statement'].tolist() + df['Standard English'].tolist()
#     labels = [0]*len(df) + [1]*len(df)

#     # Initialize tokenizer & pre-tokenize
#     tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
#     encodings = tokenizer(texts, truncation=True, padding=True, max_length=128)

#     # Build Hugging Face Dataset
#     dataset = Dataset.from_dict({
#         'input_ids':      encodings['input_ids'],
#         'attention_mask': encodings['attention_mask'],
#         'labels':         labels
#     })

#     # Load model & freeze encoder layers
#     model = DistilBertForSequenceClassification.from_pretrained(
#         'distilbert-base-uncased',
#         num_labels=2
#     )
#     for param in model.distilbert.parameters():
#         param.requires_grad = False

#     # Data collator for dynamic padding
#     data_collator = DataCollatorWithPadding(tokenizer)

#     # Metric computation function
#     def compute_metrics(p):
#         preds = p.predictions.argmax(-1)
#         acc   = accuracy_score(p.label_ids, preds)
#         prec, rec, f1, _ = precision_recall_fscore_support(
#             p.label_ids, preds, average='binary'
#         )
#         return {
#             'accuracy':  acc,
#             'precision': prec,
#             'recall':    rec,
#             'f1':        f1
#         }

#     # Prepare lists to collect per-fold metrics
#     accuracy_scores  = []
#     precision_scores = []
#     recall_scores    = []
#     f1_scores        = []

#     # 5-fold CV, 1 epoch per fold for speed
#     cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
#     for fold, (train_idx, val_idx) in enumerate(cv.split(texts, labels), start=1):
#         print(f"=== Fold {fold}/5 ===")
#         train_ds = dataset.select(train_idx)
#         val_ds   = dataset.select(val_idx)

#         training_args = TrainingArguments(
#             output_dir=f'./cv_fold{fold}',
#             num_train_epochs=1,
#             per_device_train_batch_size=16,
#             per_device_eval_batch_size=32,
#             eval_strategy='epoch',
#             logging_strategy='epoch',
#             save_strategy='no',
#             fp16=True,               # set to False if no GPU
#             report_to=['none'],
#             seed=42
#         )

#         trainer = Trainer(
#             model=model,
#             args=training_args,
#             train_dataset=train_ds,
#             eval_dataset=val_ds,
#             data_collator=data_collator,
#             compute_metrics=compute_metrics
#         )

#         trainer.train()
#         metrics = trainer.evaluate()

#         # Collect metrics
#         accuracy_scores.append(metrics['eval_accuracy'])
#         precision_scores.append(metrics['eval_precision'])
#         recall_scores.append(metrics['eval_recall'])
#         f1_scores.append(metrics['eval_f1'])

#         # Print per-fold results
#         print(f"Fold {fold}: "
#               f"Acc {metrics['eval_accuracy']:.4f}, "
#               f"Prec {metrics['eval_precision']:.4f}, "
#               f"Rec {metrics['eval_recall']:.4f}, "
#               f"F1 {metrics['eval_f1']:.4f}")

#     # Summarize CV
#     def report(name, vals):
#         mean, std = np.mean(vals), np.std(vals)
#         print(f"{name}: {mean:.4f} ± {std:.4f}")

#     print("\n=== CV summary over 5 folds ===")
#     report("Accuracy",  accuracy_scores)
#     report("Precision", precision_scores)
#     report("Recall",    recall_scores)
#     report("F1-score",  f1_scores)

#     # Retrain head on full dataset
#     print("\nRetraining classifier head on the full dataset…")
#     full_args = TrainingArguments(
#         output_dir='./full_train',
#         num_train_epochs=2,
#         per_device_train_batch_size=16,
#         save_strategy='no',
#         fp16=True,
#         report_to=['none']
#     )
#     trainer_full = Trainer(
#         model=model,
#         args=full_args,
#         train_dataset=dataset,
#         data_collator=data_collator,
#         compute_metrics=compute_metrics
#     )
#     trainer_full.train()

#     # Save the fine-tuned model & tokenizer
#     model.save_pretrained(MODEL_DIR)
#     tokenizer.save_pretrained(MODEL_DIR)
#     print(f"Saved model in '{MODEL_DIR}'")


In [16]:
# # 1) Load your saved model & tokenizer
# MODEL_DIR = 'distilbert_saved_model'
# tokenizer = DistilBertTokenizerFast.from_pretrained(MODEL_DIR)
# model     = DistilBertForSequenceClassification.from_pretrained(MODEL_DIR)
# model.eval()

# # 2) Prompt the user for input
# sentence = input("Enter a sentence to check (grammar):\n> ")

# # 3) Tokenize & forward pass
# enc = tokenizer([sentence], padding=True, truncation=True, return_tensors='pt', max_length=128)
# with torch.no_grad():
#     logits = model(**enc).logits

# # 4) Convert to probabilities & predicted label
# probs = torch.softmax(logits, dim=-1)[0]
# pred  = probs.argmax().item()
# label = "Correct" if pred == 1 else "Ungrammatical"
# score = probs[pred].item()

# # 5) Print result
# print(f"\n\"{sentence}\" → {label} (confidence: {score:.2f})")

#T5 Model


In [20]:
# import os
# import pandas as pd
# from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
# from datasets import Dataset
# import torch
# from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction


# model_name = "t5-small"
# model_dir = "./t5-grammar-correction"

# # Load tokenizer and model
# tokenizer = T5Tokenizer.from_pretrained(model_name)

# if os.path.exists(model_dir):
#     print("Saved model found. Loading the model...")
#     model = T5ForConditionalGeneration.from_pretrained(model_dir)
# else:
#     print("No saved model found. Training model from scratch...")
#     model = T5ForConditionalGeneration.from_pretrained(model_name)

#     # Prepare input and target sequences
#     inputs = ["fix: " + text for text in df['Ungrammatical Statement']]
#     targets = df['Standard English'].tolist()

#     # Tokenize input and output
#     input_encodings = tokenizer(inputs, padding="max_length", truncation=True, max_length=64, return_tensors="pt")
#     target_encodings = tokenizer(targets, padding="max_length", truncation=True, max_length=64, return_tensors="pt")

#     labels = target_encodings["input_ids"]
#     labels[labels == tokenizer.pad_token_id] = -100

#     dataset = Dataset.from_dict({
#         'input_ids': input_encodings['input_ids'],
#         'attention_mask': input_encodings['attention_mask'],
#         'labels': labels
#     })

#     os.environ["WANDB_DISABLED"] = "true"

#     training_args = TrainingArguments(
#         output_dir=model_dir,
#         per_device_train_batch_size=8,
#         per_device_eval_batch_size=8,
#         num_train_epochs=4,
#         logging_dir="./logs",
#         save_steps=500,
#         save_total_limit=2
#     )

#     trainer = Trainer(
#         model=model,
#         args=training_args,
#         train_dataset=dataset,
#         eval_dataset=dataset,
#         # tokenizer=tokenizer  # Optional to keep or remove based on warning
#     )

#     trainer.train()
#     trainer.save_model(model_dir)

# # Grammar correction function
# def correct_grammar(sentence):
#     input_text = "fix: " + sentence
#     input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=64, truncation=True)
#     output_ids = model.generate(input_ids, max_length=64, num_beams=4, early_stopping=True)
#     return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# # Evaluation and test code (same as before)...
# print("\nEvaluating model on the dataset...\n")

# smoothie = SmoothingFunction().method4
# total_bleu = 0
# correct_count = 0

# for ungram, ref in zip(df['Ungrammatical Statement'], df['Standard English']):
#     pred = correct_grammar(ungram).strip()
#     ref = ref.strip()

#     # BLEU score
#     ref_tokens = ref.split()
#     pred_tokens = pred.split()
#     bleu = sentence_bleu([ref_tokens], pred_tokens, smoothing_function=smoothie)
#     total_bleu += bleu

#     # Exact match accuracy (case-insensitive)
#     if pred.lower() == ref.lower():
#         correct_count += 1

# average_bleu = total_bleu / len(df)
# accuracy = correct_count / len(df)

# print(f"Average BLEU score: {average_bleu:.4f}")
# print(f"Exact Match Accuracy: {accuracy:.2%}")


# print(f"Average BLEU score: {average_bleu:.4f}")
# print(f"Exact Match Accuracy: {accuracy:.2%}")

# # --- Test on a new example sentence ---
# user_sentence = input("\nEnter a sentence to correct grammar:\n")
# corrected_sentence = correct_grammar(user_sentence)
# print("Corrected:", corrected_sentence)

In [23]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import torch

model_name = "t5-small"
model_dir = "./t5-grammar-correction-new"
k_folds = 5
num_train_epochs = 3
max_length = 64

tokenizer = T5Tokenizer.from_pretrained(model_name)

def report(name, vals):
    mean, std = np.mean(vals), np.std(vals)
    print(f"{name}: {mean:.4f} ± {std:.4f}")

smoothie = SmoothingFunction().method4

# Load your dataframe 'df' before this point
# df must have columns: 'Ungrammatical Statement', 'Standard English'

if os.path.exists(model_dir):
    print("Saved model found. Loading the model...")
    model = T5ForConditionalGeneration.from_pretrained(model_dir)

else:
    print("No saved model found. Starting training with cross-validation...")

    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    bleu_scores = []

    kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)
    fold_num = 1

    for train_index, val_index in kf.split(df):
        print(f"\n=== Fold {fold_num} ===")
        train_df = df.iloc[train_index].reset_index(drop=True)
        val_df = df.iloc[val_index].reset_index(drop=True)

        # Prepare datasets
        train_inputs = ["fix: " + text for text in train_df['Ungrammatical Statement']]
        train_targets = train_df['Standard English'].tolist()

        val_inputs = ["fix: " + text for text in val_df['Ungrammatical Statement']]
        val_targets = val_df['Standard English'].tolist()

        # Tokenize train
        train_input_encodings = tokenizer(train_inputs, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")
        train_target_encodings = tokenizer(train_targets, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")
        train_labels = train_target_encodings["input_ids"]
        train_labels[train_labels == tokenizer.pad_token_id] = -100

        train_dataset = Dataset.from_dict({
            'input_ids': train_input_encodings['input_ids'],
            'attention_mask': train_input_encodings['attention_mask'],
            'labels': train_labels
        })

        # Tokenize validation
        val_input_encodings = tokenizer(val_inputs, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")
        val_target_encodings = tokenizer(val_targets, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")
        val_labels = val_target_encodings["input_ids"]
        val_labels[val_labels == tokenizer.pad_token_id] = -100

        val_dataset = Dataset.from_dict({
            'input_ids': val_input_encodings['input_ids'],
            'attention_mask': val_input_encodings['attention_mask'],
            'labels': val_labels
        })

        model = T5ForConditionalGeneration.from_pretrained(model_name)

        training_args = TrainingArguments(
            output_dir=f"./t5-cv-fold{fold_num}",
            per_device_train_batch_size=8,
            per_device_eval_batch_size=8,
            num_train_epochs=num_train_epochs,
            logging_dir="./logs",
            save_steps=500,
            save_total_limit=1,
            # evaluation_strategy="epoch",
            # save_strategy="epoch",
            # load_best_model_at_end=True,
            # metric_for_best_model="eval_loss",
            # greater_is_better=False,
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
        )

        trainer.train()

        # Evaluate on validation set
        model.eval()
        predictions = []
        references = []

        for sentence in val_df['Ungrammatical Statement']:
            input_text = "fix: " + sentence
            input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=max_length, truncation=True)
            with torch.no_grad():
                outputs = model.generate(input_ids, max_length=max_length, num_beams=4, early_stopping=True)
            pred = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
            predictions.append(pred)

        references = val_df['Standard English'].str.strip().tolist()

        # Calculate BLEU and exact match accuracy
        bleu_sum = 0
        correct_exact = 0
        for pred, ref in zip(predictions, references):
            pred_tokens = pred.split()
            ref_tokens = ref.split()
            bleu = sentence_bleu([ref_tokens], pred_tokens, smoothing_function=smoothie)
            bleu_sum += bleu
            if pred.lower() == ref.lower():
                correct_exact += 1
        bleu_score = bleu_sum / len(predictions)
        accuracy = correct_exact / len(predictions)

        # Precision, Recall, F1 micro (treat exact matches as 1, else 0)
        y_true = [1] * len(references)  # all correct labels
        y_pred = [1 if p.lower() == r.lower() else 0 for p, r in zip(predictions, references)]

        precision, recall, f1, _ = precision_recall_fscore_support(
            y_true, y_pred, average='micro'
        )

        print(f"Fold {fold_num}: BLEU={bleu_score:.4f}, Accuracy={accuracy:.4f}, "
              f"Precision={precision:.4f}, Recall={recall:.4f}, F1={f1:.4f}")

        accuracy_scores.append(accuracy)
        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)
        bleu_scores.append(bleu_score)

        fold_num += 1

    print("\n=== Cross-validation summary ===")
    report("BLEU", bleu_scores)
    report("Accuracy", accuracy_scores)
    report("Precision", precision_scores)
    report("Recall", recall_scores)
    report("F1-score", f1_scores)

    # Final training on full dataset
    print("\nTraining on full dataset for final model...")
    inputs = ["fix: " + text for text in df['Ungrammatical Statement']]
    targets = df['Standard English'].tolist()

    input_encodings = tokenizer(inputs, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")
    target_encodings = tokenizer(targets, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")

    labels = target_encodings["input_ids"]
    labels[labels == tokenizer.pad_token_id] = -100

    full_dataset = Dataset.from_dict({
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': labels
    })

    model = T5ForConditionalGeneration.from_pretrained(model_name)
    training_args = TrainingArguments(
        output_dir=model_dir,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=num_train_epochs,
        logging_dir="./logs",
        save_steps=500,
        save_total_limit=2,
    )
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=full_dataset,
        eval_dataset=full_dataset,
    )

    trainer.train()
    trainer.save_model(model_dir)

# Grammar correction function using final trained or loaded model
def correct_grammar(sentence):
    input_text = "fix: " + sentence
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=max_length, truncation=True)
    output_ids = model.generate(input_ids, max_length=max_length, num_beams=4, early_stopping=True)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# User input for correction
user_sentence = input("\nEnter a sentence to correct grammar:\n")
corrected_sentence = correct_grammar(user_sentence)
print("Corrected:", corrected_sentence)


No saved model found. Starting training with cross-validation...

=== Fold 1 ===


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss
500,0.633000


Fold 1: BLEU=0.6675, Accuracy=0.3540, Precision=0.3540, Recall=0.3540, F1=0.3540

=== Fold 2 ===


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


...

Step,Training Loss
500,0.587700


Fold 5: BLEU=0.6624, Accuracy=0.3499, Precision=0.3499, Recall=0.3499, F1=0.3499

=== Cross-validation summary ===
BLEU: 0.6686 ± 0.0053
Accuracy: 0.3459 ± 0.0198
Precision: 0.3459 ± 0.0198
Recall: 0.3459 ± 0.0198
F1-score: 0.3459 ± 0.0198

Training on full dataset for final model...


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss
500,0.611800



Enter a sentence to correct grammar:
she are running late
Corrected: she is running late
